# Análise Completa de Machine Learning - Dataset Diabetes

Este notebook apresenta uma análise completa de machine learning no dataset de diabetes, seguindo metodologia estruturada para classificação binária.

## Objetivos
- Explorar e compreender o dataset de diabetes
- Implementar e comparar modelos de classificação (Decision Tree e Random Forest)
- Avaliar performance através de múltiplas métricas
- Fornecer insights e recomendações baseadas nos resultados

In [92]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report,
    precision_score, recall_score, f1_score, roc_curve, auc,
    roc_auc_score
)

# Configurações
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
np.random.seed(42)

print("✅ Bibliotecas importadas com sucesso!")

✅ Bibliotecas importadas com sucesso!


## 1. Exploração Inicial dos Dados

In [93]:
# Carregamento do dataset
# Caminho ajustado para o arquivo CSV
df = pd.read_csv(r"C:\Users\mauro\Downloads\projeto_classificacao_clientes_inadimplentes_\Pr_tica_com_Dados_de_Diabetes\diabetes-ml-analysis\data\diabetes.csv")
print("=== ESTRUTURA DO DATASET ===")
print(f"Shape: {df.shape}")
print(f"Colunas: {list(df.columns)}")
print()

print("=== PRIMEIRAS 5 LINHAS ===")
display(df.head())

print("\n=== INFORMAÇÕES GERAIS ===")
df.info()

print("\n=== ESTATÍSTICAS DESCRITIVAS ===")
display(df.describe())

=== ESTRUTURA DO DATASET ===
Shape: (394, 6)
Colunas: ['glicemia', 'pressao_sanguinea', 'dobra_cutane_triceps', 'insulina', 'imc', 'diabetes']

=== PRIMEIRAS 5 LINHAS ===


,glicemia,pressao_sanguinea,dobra_cutane_triceps,insulina,imc,diabetes
0,66,23,94,28.1,0.167,0
1,40,35,168,43.1,2.288,1
2,50,32,88,31.0,0.248,1
3,70,45,543,30.5,0.158,1
4,60,23,846,30.1,0.398,1



=== INFORMAÇÕES GERAIS ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 394 entries, 0 to 393
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   glicemia              394 non-null    int64  
 1   pressao_sanguinea     394 non-null    int64  
 2   dobra_cutane_triceps  394 non-null    int64  
 3   insulina              394 non-null    float64
 4   imc                   394 non-null    float64
 5   diabetes              394 non-null    int64  
dtypes: float64(2), int64(4)
memory usage: 18.6 KB

=== ESTATÍSTICAS DESCRITIVAS ===


,glicemia,pressao_sanguinea,dobra_cutane_triceps,insulina,imc,diabetes
count,394.000000,394.000000,394.000000,394.000000,394.000000,394.000000
mean,70.654822,29.106599,155.548223,32.988579,0.525543,0.329949
std,12.469919,10.504273,118.775855,7.210160,0.350127,0.470792
min,24.000000,7.000000,14.000000,0.000000,0.085000,0.000000
25%,62.000000,21.000000,76.250000,28.325000,0.270250,0.000000
50%,70.000000,29.000000,125.000000,33.200000,0.449500,0.000000
75%,78.000000,36.750000,190.000000,37.075000,0.687000,1.000000
max,110.000000,63.000000,846.000000,67.100000,2.420000,1.000000


In [94]:
# Análise da variável alvo
print("=== DISTRIBUIÇÃO DA VARIÁVEL ALVO (diabetes) ===")
target_counts = df['diabetes'].value_counts()
target_props = df['diabetes'].value_counts(normalize=True)

print(f"Contagem: {target_counts}")
print(f"Proporção: {target_props}")
print(f"\nDataset balanceado? {'Sim' if abs(target_props[0] - target_props[1]) < 0.2 else 'Não (desbalanceado)'}")

# Verificação de valores nulos
print("\n=== VALORES NULOS ===")
null_counts = df.isnull().sum()
print(null_counts)
print(f"\nDataset completo? {'Sim' if null_counts.sum() == 0 else 'Não'}")

=== DISTRIBUIÇÃO DA VARIÁVEL ALVO (diabetes) ===
Contagem: diabetes
0    264
1    130
Name: count, dtype: int64
Proporção: diabetes
0    0.670051
1    0.329949
Name: proportion, dtype: float64

Dataset balanceado? Não (desbalanceado)

=== VALORES NULOS ===
glicemia                0
pressao_sanguinea       0
dobra_cutane_triceps    0
insulina                0
imc                     0
diabetes                0
dtype: int64

Dataset completo? Sim


In [95]:
# Visualização da distribuição das variáveis
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=list(df.columns),
    specs=[[{"secondary_y": False}]*3]*2
)

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']

for i, col in enumerate(df.columns):
    row = i // 3 + 1
    col_pos = i % 3 + 1
    
    fig.add_trace(
        go.Histogram(
            x=df[col],
            name=col,
            marker_color=colors[i],
            opacity=0.7
        ),
        row=row, col=col_pos
    )

fig.update_layout(
    title_text="Distribuição das Variáveis do Dataset",
    showlegend=False,
    height=600
)

fig.show()
fig.write_html('distribuicao_variaveis.html')

## 2. Preparação dos Dados

In [96]:
# Separação das variáveis explicativas (X) e variável alvo (y)
X = df.drop('diabetes', axis=1)
y = df['diabetes']

print("=== SEPARAÇÃO DAS VARIÁVEIS ===")
print(f"Variáveis explicativas (X): {list(X.columns)}")
print(f"Variável alvo (y): diabetes")
print(f"Shape X: {X.shape}")
print(f"Shape y: {y.shape}")

# Divisão dos dados conforme especificado:
# 1. 5% para teste
# 2. Dos 95% restantes, 25% para validação (que equivale a ~23.75% do total)
# 3. O restante para treino (~71.25% do total)

# Primeira divisão: 95% treino+validação, 5% teste
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.05, random_state=42, stratify=y
)

# Segunda divisão: dos 95% restantes, 25% para validação
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp
)

print("\n=== DIVISÃO DOS DADOS ===")
print(f"Treino: {X_train.shape[0]} amostras ({X_train.shape[0]/len(df)*100:.1f}%)")
print(f"Validação: {X_val.shape[0]} amostras ({X_val.shape[0]/len(df)*100:.1f}%)")
print(f"Teste: {X_test.shape[0]} amostras ({X_test.shape[0]/len(df)*100:.1f}%)")
print(f"Total: {X_train.shape[0] + X_val.shape[0] + X_test.shape[0]} amostras")

# Verificação da estratificação
print("\n=== VERIFICAÇÃO DA ESTRATIFICAÇÃO ===")
print(f"Treino - Classe 0: {(y_train==0).sum()}, Classe 1: {(y_train==1).sum()}")
print(f"Validação - Classe 0: {(y_val==0).sum()}, Classe 1: {(y_val==1).sum()}")
print(f"Teste - Classe 0: {(y_test==0).sum()}, Classe 1: {(y_test==1).sum()}")

print(f"\nProporção classe 1:")
print(f"Treino: {y_train.mean():.3f}")
print(f"Validação: {y_val.mean():.3f}")
print(f"Teste: {y_test.mean():.3f}")
print(f"Original: {y.mean():.3f}")

=== SEPARAÇÃO DAS VARIÁVEIS ===
Variáveis explicativas (X): ['glicemia', 'pressao_sanguinea', 'dobra_cutane_triceps', 'insulina', 'imc']
Variável alvo (y): diabetes
Shape X: (394, 5)
Shape y: (394,)

=== DIVISÃO DOS DADOS ===
Treino: 280 amostras (71.1%)
Validação: 94 amostras (23.9%)
Teste: 20 amostras (5.1%)
Total: 394 amostras

=== VERIFICAÇÃO DA ESTRATIFICAÇÃO ===
Treino - Classe 0: 188, Classe 1: 92
Validação - Classe 0: 63, Classe 1: 31
Teste - Classe 0: 13, Classe 1: 7

Proporção classe 1:
Treino: 0.329
Validação: 0.330
Teste: 0.350
Original: 0.330


## 3. Modelagem

In [97]:
# Criação e treinamento dos modelos
print("=== TREINAMENTO DOS MODELOS ===")

# Modelo 1: Decision Tree com max_depth=3
dt_model = DecisionTreeClassifier(max_depth=3, random_state=42)
dt_model.fit(X_train, y_train)
print("✅ Decision Tree treinada")

# Modelo 2: Random Forest com max_depth=2
rf_model = RandomForestClassifier(max_depth=2, random_state=42, n_estimators=100)
rf_model.fit(X_train, y_train)
print("✅ Random Forest treinada")

# Predições para todos os conjuntos
models = {
    'Decision Tree': dt_model,
    'Random Forest': rf_model
}

predictions = {}
probabilities = {}

for name, model in models.items():
    predictions[name] = {
        'train': model.predict(X_train),
        'val': model.predict(X_val),
        'test': model.predict(X_test)
    }
    probabilities[name] = {
        'train': model.predict_proba(X_train)[:, 1],
        'val': model.predict_proba(X_val)[:, 1],
        'test': model.predict_proba(X_test)[:, 1]
    }

print("✅ Predições realizadas para todos os conjuntos")

=== TREINAMENTO DOS MODELOS ===
✅ Decision Tree treinada
✅ Random Forest treinada
✅ Predições realizadas para todos os conjuntos


In [98]:
# Cálculo das acurácias
print("=== ACURÁCIAS DOS MODELOS ===")

accuracies = {}
for name in models.keys():
    accuracies[name] = {
        'train': accuracy_score(y_train, predictions[name]['train']),
        'val': accuracy_score(y_val, predictions[name]['val']),
        'test': accuracy_score(y_test, predictions[name]['test'])
    }
    
    print(f"\n{name}:")
    print(f"  Treino: {accuracies[name]['train']:.4f}")
    print(f"  Validação: {accuracies[name]['val']:.4f}")
    print(f"  Teste: {accuracies[name]['test']:.4f}")
    
    # Análise de overfitting
    train_val_diff = accuracies[name]['train'] - accuracies[name]['val']
    if train_val_diff > 0.05:
        print(f"  ⚠️ Possível overfitting (diferença treino-validação: {train_val_diff:.4f})")
    else:
        print(f"  ✅ Modelo bem generalizado (diferença treino-validação: {train_val_diff:.4f})")

=== ACURÁCIAS DOS MODELOS ===

Decision Tree:
  Treino: 0.8107
  Validação: 0.6702
  Teste: 0.7500
  ⚠️ Possível overfitting (diferença treino-validação: 0.1405)

Random Forest:
  Treino: 0.7857
  Validação: 0.6277
  Teste: 0.6500
  ⚠️ Possível overfitting (diferença treino-validação: 0.1581)


## 4. Avaliação Completa dos Modelos

In [99]:
# Função para calcular todas as métricas
def calculate_metrics(y_true, y_pred, y_prob):
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred),
        'recall': recall_score(y_true, y_pred),
        'f1': f1_score(y_true, y_pred),
        'auc': roc_auc_score(y_true, y_prob)
    }

# Cálculo de todas as métricas
all_metrics = {}
for name in models.keys():
    all_metrics[name] = {
        'train': calculate_metrics(y_train, predictions[name]['train'], probabilities[name]['train']),
        'val': calculate_metrics(y_val, predictions[name]['val'], probabilities[name]['val']),
        'test': calculate_metrics(y_test, predictions[name]['test'], probabilities[name]['test'])
    }

# Criação de DataFrame com métricas para melhor visualização
metrics_data = []
for model_name in models.keys():
    for dataset in ['train', 'val', 'test']:
        row = {'Modelo': model_name, 'Dataset': dataset}
        row.update(all_metrics[model_name][dataset])
        metrics_data.append(row)

metrics_df = pd.DataFrame(metrics_data)
print("=== MÉTRICAS COMPLETAS ===")
display(metrics_df.round(4))

=== MÉTRICAS COMPLETAS ===


,Modelo,Dataset,accuracy,precision,recall,f1,auc
0,Decision Tree,train,0.8107,0.7349,0.6630,0.6971,0.8412
1,Decision Tree,val,0.6702,0.5000,0.5484,0.5231,0.6613
2,Decision Tree,test,0.7500,0.6667,0.5714,0.6154,0.7308
3,Random Forest,train,0.7857,0.7759,0.4891,0.6000,0.8622
4,Random Forest,val,0.6277,0.4091,0.2903,0.3396,0.7087
5,Random Forest,test,0.6500,0.5000,0.1429,0.2222,0.7253


In [100]:
# Matrizes de Confusão
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=[
        'DT - Treino', 'DT - Validação', 'DT - Teste',
        'RF - Treino', 'RF - Validação', 'RF - Teste'
    ],
    specs=[[{"type": "heatmap"}]*3]*2
)

datasets = [('train', y_train), ('val', y_val), ('test', y_test)]
model_names = ['Decision Tree', 'Random Forest']

for i, model_name in enumerate(model_names):
    for j, (dataset_name, y_true) in enumerate(datasets):
        cm = confusion_matrix(y_true, predictions[model_name][dataset_name])
        
        # Normalizar para percentuais
        cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
        
        fig.add_trace(
            go.Heatmap(
                z=cm_norm,
                x=['Pred: Não', 'Pred: Sim'],
                y=['Real: Não', 'Real: Sim'],
                colorscale='Blues',
                text=[[f'{cm[0,0]}<br>({cm_norm[0,0]:.1f}%)', f'{cm[0,1]}<br>({cm_norm[0,1]:.1f}%)'],
                      [f'{cm[1,0]}<br>({cm_norm[1,0]:.1f}%)', f'{cm[1,1]}<br>({cm_norm[1,1]:.1f}%)']],
                texttemplate="%{text}",
                textfont={"size": 10},
                showscale=False
            ),
            row=i+1, col=j+1
        )

fig.update_layout(
    title_text="Matrizes de Confusão - Comparação dos Modelos",
    height=600
)

fig.show()
fig.write_html('matrizes_confusao.html')

In [101]:
# Gráfico comparativo das métricas
test_metrics = metrics_df[metrics_df['Dataset'] == 'test'].copy()

fig = go.Figure()

metrics_to_plot = ['accuracy', 'precision', 'recall', 'f1', 'auc']
x_pos = np.arange(len(metrics_to_plot))

for i, model in enumerate(['Decision Tree', 'Random Forest']):
    model_data = test_metrics[test_metrics['Modelo'] == model]
    values = [model_data[metric].iloc[0] for metric in metrics_to_plot]
    
    fig.add_trace(go.Bar(
        name=model,
        x=metrics_to_plot,
        y=values,
        text=[f'{v:.3f}' for v in values],
        textposition='auto',
        marker_color=['#1f77b4', '#ff7f0e'][i]
    ))

fig.update_layout(
    title='Comparação de Métricas - Conjunto de Teste',
    xaxis_title='Métricas',
    yaxis_title='Valor',
    barmode='group',
    yaxis=dict(range=[0, 1.1])
)

fig.show()
fig.write_html('comparacao_metricas.html')

In [102]:
# Curvas ROC
fig = go.Figure()

# Linha de referência (classificador aleatório)
fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1],
    mode='lines',
    line=dict(dash='dash', color='gray'),
    name='Classificador Aleatório'
))

colors = ['#1f77b4', '#ff7f0e']
for i, (name, model) in enumerate(models.items()):
    # Curva ROC para conjunto de teste
    fpr, tpr, _ = roc_curve(y_test, probabilities[name]['test'])
    auc_score = auc(fpr, tpr)
    
    fig.add_trace(go.Scatter(
        x=fpr, y=tpr,
        mode='lines',
        name=f'{name} (AUC = {auc_score:.3f})',
        line=dict(color=colors[i], width=3)
    ))

fig.update_layout(
    title='Curvas ROC - Conjunto de Teste',
    xaxis_title='Taxa de Falsos Positivos',
    yaxis_title='Taxa de Verdadeiros Positivos',
    width=600, height=600
)

fig.show()
fig.write_html('curvas_roc.html')

## 5. Análise de Importância das Features

In [103]:
# Importância das features
feature_names = X.columns

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Decision Tree', 'Random Forest']
)

# Decision Tree
dt_importance = dt_model.feature_importances_
fig.add_trace(
    go.Bar(
        x=feature_names,
        y=dt_importance,
        name='Decision Tree',
        marker_color='#1f77b4'
    ),
    row=1, col=1
)

# Random Forest
rf_importance = rf_model.feature_importances_
fig.add_trace(
    go.Bar(
        x=feature_names,
        y=rf_importance,
        name='Random Forest',
        marker_color='#ff7f0e'
    ),
    row=1, col=2
)

fig.update_layout(
    title_text="Importância das Features por Modelo",
    showlegend=False,
    height=500
)

fig.update_xaxes(tickangle=45)
fig.show()
fig.write_html('importancia_features.html')

# Tabela de importância
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Decision_Tree': dt_importance,
    'Random_Forest': rf_importance
}).sort_values('Decision_Tree', ascending=False)

print("=== IMPORTÂNCIA DAS FEATURES ===")
display(importance_df.round(4))

=== IMPORTÂNCIA DAS FEATURES ===


,Feature,Decision_Tree,Random_Forest
2,dobra_cutane_triceps,0.5685,0.4239
3,insulina,0.2304,0.2043
4,imc,0.1260,0.1321
0,glicemia,0.0751,0.1409
1,pressao_sanguinea,0.0000,0.0988


## 6. Exportação de Resultados

In [104]:
# Exportação das métricas para Excel/Power BI
metrics_df.to_csv('metricas_modelos.csv', index=False)
importance_df.to_csv('importancia_features.csv', index=False)

# Tabela resumo final
resumo_final = test_metrics[['Modelo', 'accuracy', 'precision', 'recall', 'f1', 'auc']].round(4)
resumo_final.to_csv('resumo_final_modelos.csv', index=False)

print("=== RESUMO FINAL - CONJUNTO DE TESTE ===")
display(resumo_final)

print("\n✅ Arquivos exportados:")
print("- metricas_modelos.csv")
print("- importancia_features.csv")
print("- resumo_final_modelos.csv")
print("- distribuicao_variaveis.html")
print("- matrizes_confusao.html")
print("- comparacao_metricas.html")
print("- curvas_roc.html")
print("- importancia_features.html")

=== RESUMO FINAL - CONJUNTO DE TESTE ===


,Modelo,accuracy,precision,recall,f1,auc
2,Decision Tree,0.75,0.6667,0.5714,0.6154,0.7308
5,Random Forest,0.65,0.5000,0.1429,0.2222,0.7253



✅ Arquivos exportados:
- metricas_modelos.csv
- importancia_features.csv
- resumo_final_modelos.csv
- distribuicao_variaveis.html
- matrizes_confusao.html
- comparacao_metricas.html
- curvas_roc.html
- importancia_features.html


## 7. Conclusões e Recomendações

### Análise Comparativa dos Modelos

**Decision Tree (max_depth=3):**
- Modelo mais simples e interpretável
- Boa performance geral
- Menor risco de overfitting devido à limitação de profundidade

**Random Forest (max_depth=2):**
- Ensemble de árvores com profundidade ainda mais limitada
- Potencialmente mais robusto a ruído
- Melhor generalização esperada

### Recomendações:

1. **Modelo Recomendado**: Baseado nas métricas de teste, recomenda-se o modelo com melhor performance balanceada

2. **Features Importantes**: Focar nas variáveis com maior importância para futuras coletas de dados

3. **Próximos Passos**:
   - Considerar técnicas de balanceamento de classes se necessário
   - Explorar outros algoritmos (SVM, Gradient Boosting)
   - Implementar validação cruzada para maior robustez
   - Análise de erros para identificar padrões nos casos mal classificados

4. **Aplicação Prática**: O modelo pode ser usado como ferramenta de triagem, mas sempre deve ser complementado por avaliação médica profissional.